In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'

In [77]:
all_df = pd.read_csv(root+R'\all_heatmap.csv')
#all_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

In [78]:
sig_only = all_df.loc[all_df['P_Value'] < 0.05]
vc = sig_only.Cancer.value_counts()
df = vc.to_frame()
df = df.reset_index()
df = df.rename(columns = {'index': 'Cancer','Cancer':'Sig_One'})

# Get a list of significant genes in at least one cancer

In [82]:
# Uses corrected pvals from wrap_ttest, each cancer's cutoff is 0.05
sig_one_cancer = {}
for cancer in df.Cancer:
    c_df = sig_only.loc[sig_only['Cancer'] == cancer]
    c_prot_list = list(c_df.Proteomics)
    sig_one_cancer[cancer] = c_prot_list 
    print(cancer, len(c_prot_list), '\n')
print('No sig for Brca and Colon')  

Gbm 1900 

Hnscc 570 

En 415 

Luad 111 

Lscc 3 

Ov 1 

No sig for Brca and Colon


In [118]:
# Get dictionary with lists of genes sig in two cancers
two = {}
two_len = {}
cancer_list = ['Gbm', 'Hnscc', 'Luad', 'Lscc', 'Ov', 'En']
for c1 in sig_one_cancer:
    cancer_list.remove(c1)
    for c2 in cancer_list:
        #print(c1, c2, '\n')
        c1_genes = sig_one_cancer[c1]
        c2_genes = sig_one_cancer[c2]
        in_both = list(set(c1_genes) & set(c2_genes))
        two[c1+'_'+c2] = in_both
        two_len[c1+'_'+c2] = len(in_both)
        #print(c1+'_'+c2, len(in_both))
        
two['Gbm_Ov']
df_2 = pd.DataFrame.from_dict(two_len, orient = 'index')
df_2

,0
Gbm_Hnscc,164
Gbm_Luad,38
Gbm_Lscc,2
Gbm_Ov,1
Gbm_En,111
Hnscc_Luad,38
Hnscc_Lscc,1
Hnscc_Ov,0
Hnscc_En,50
En_Luad,1


In [102]:
# Check
all_df.loc[all_df['Proteomics'] == 'MMS19']

,Proteomics,P_Value,Medians,Cancer
66,MMS19,0.000629,-0.324287,Gbm
12471,MMS19,0.145010,-0.123084,Hnscc
22365,MMS19,0.027361,-0.620000,Luad
33043,MMS19,0.366903,-0.356650,Lscc
45093,MMS19,0.403055,-0.653400,Brca
54708,MMS19,0.000110,-0.324897,Ov
69516,MMS19,0.519462,-0.084700,En
76038,MMS19,0.624087,-0.121800,Colon


In [108]:
# Get dictionary with lists of genes sig in two cancers
three = {}
c2_list = ['Gbm', 'Hnscc', 'Luad', 'Lscc', 'Ov', 'En']
c3_list = ['Hnscc', 'Luad', 'Lscc', 'Ov', 'En']
for c1 in sig_one_cancer:
    c2_list.remove(c1)
    for c2 in c2_list:
        c3_list.remove(c2)
        for c3 in c3_list:
            #print(c1, c2, '\n')
            c1_genes = sig_one_cancer[c1]
            c2_genes = sig_one_cancer[c2]
            c3_genes = sig_one_cancer[c3]
            in_all = list(set(c1_genes) & set(c2_genes) & set(c3_genes))
            three[c1+'_'+c2+'_'+c3] = in_all
            print(c1+'_'+c2+'_'+c3, len(in_all))
        
three

Gbm_Hnscc_Luad 14
Gbm_Hnscc_Lscc 1
Gbm_Hnscc_Ov 0
Gbm_Hnscc_En 21
Gbm_Luad_Lscc 0
Gbm_Luad_Ov 1
Gbm_Luad_En 1
Gbm_Lscc_Ov 0
Gbm_Lscc_En 0
Gbm_Ov_En 0


ValueError: list.remove(x): x not in list

# Get a list of significant genes in multiple cancers

In [21]:
# sig in multiple (more than 1) cancers
mult = list(set(i for i in flat_list if flat_list.count(i) > 1)) # Keep genes sig in more than 1 cancer
print('Number of significant in mult cancers:', len(mult))

Number of significant in mult cancers: 333
